<a href="https://colab.research.google.com/github/Kvnhooman/AAI530_Final_Project_Team7/blob/main/MSAAI530_Team_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets huggingface_hub pyarrow pandas numpy


In [ ]:
import numpy as np
import pandas as pd

from datasets import load_dataset

import math
from collections import Counter
from tqdm.auto import tqdm


## Load the dataset from HF

In [ ]:
ds = load_dataset("witgaw/METR-LA")  # downloads parquet shards + builds splits
ds


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

val.parquet:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/43.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4962618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/708975 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1417950 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['node_id', 't0_timestamp', 'x_t-11_d0', 'x_t-11_d1', 'x_t-10_d0', 'x_t-10_d1', 'x_t-9_d0', 'x_t-9_d1', 'x_t-8_d0', 'x_t-8_d1', 'x_t-7_d0', 'x_t-7_d1', 'x_t-6_d0', 'x_t-6_d1', 'x_t-5_d0', 'x_t-5_d1', 'x_t-4_d0', 'x_t-4_d1', 'x_t-3_d0', 'x_t-3_d1', 'x_t-2_d0', 'x_t-2_d1', 'x_t-1_d0', 'x_t-1_d1', 'x_t+0_d0', 'x_t+0_d1', 'y_t+1_d0', 'y_t+1_d1', 'y_t+2_d0', 'y_t+2_d1', 'y_t+3_d0', 'y_t+3_d1', 'y_t+4_d0', 'y_t+4_d1', 'y_t+5_d0', 'y_t+5_d1', 'y_t+6_d0', 'y_t+6_d1', 'y_t+7_d0', 'y_t+7_d1', 'y_t+8_d0', 'y_t+8_d1', 'y_t+9_d0', 'y_t+9_d1', 'y_t+10_d0', 'y_t+10_d1', 'y_t+11_d0', 'y_t+11_d1', 'y_t+12_d0', 'y_t+12_d1'],
        num_rows: 4962618
    })
    validation: Dataset({
        features: ['node_id', 't0_timestamp', 'x_t-11_d0', 'x_t-11_d1', 'x_t-10_d0', 'x_t-10_d1', 'x_t-9_d0', 'x_t-9_d1', 'x_t-8_d0', 'x_t-8_d1', 'x_t-7_d0', 'x_t-7_d1', 'x_t-6_d0', 'x_t-6_d1', 'x_t-5_d0', 'x_t-5_d1', 'x_t-4_d0', 'x_t-4_d1', 'x_t-3_d0', 'x_t-3_d1', 'x_t-2_

In [ ]:
ds.keys()
ds["train"].num_rows
ds["train"].features


{'node_id': Value('int64'),
 't0_timestamp': Value('string'),
 'x_t-11_d0': Value('float64'),
 'x_t-11_d1': Value('float64'),
 'x_t-10_d0': Value('float64'),
 'x_t-10_d1': Value('float64'),
 'x_t-9_d0': Value('float64'),
 'x_t-9_d1': Value('float64'),
 'x_t-8_d0': Value('float64'),
 'x_t-8_d1': Value('float64'),
 'x_t-7_d0': Value('float64'),
 'x_t-7_d1': Value('float64'),
 'x_t-6_d0': Value('float64'),
 'x_t-6_d1': Value('float64'),
 'x_t-5_d0': Value('float64'),
 'x_t-5_d1': Value('float64'),
 'x_t-4_d0': Value('float64'),
 'x_t-4_d1': Value('float64'),
 'x_t-3_d0': Value('float64'),
 'x_t-3_d1': Value('float64'),
 'x_t-2_d0': Value('float64'),
 'x_t-2_d1': Value('float64'),
 'x_t-1_d0': Value('float64'),
 'x_t-1_d1': Value('float64'),
 'x_t+0_d0': Value('float64'),
 'x_t+0_d1': Value('float64'),
 'y_t+1_d0': Value('float64'),
 'y_t+1_d1': Value('float64'),
 'y_t+2_d0': Value('float64'),
 'y_t+2_d1': Value('float64'),
 'y_t+3_d0': Value('float64'),
 'y_t+3_d1': Value('float64'),
 'y_

# As this is a large dataset, we will initially truncate to 200,000 datapoints to inspect and determine some rough summary statistics for the subset.

In [ ]:
N = 200_000  # set truncated dataset size
df = ds["train"].select(range(N)).to_pandas()

df["t0_timestamp"] = pd.to_datetime(df["t0_timestamp"], errors="coerce")
df.head(), df.shape


(   node_id        t0_timestamp  x_t-11_d0  x_t-11_d1  x_t-10_d0  x_t-10_d1  \
 0        0 2012-03-01 00:55:00     64.375        0.0  62.666667   0.003472   
 1        1 2012-03-01 00:55:00     67.625        0.0  68.555556   0.003472   
 2        2 2012-03-01 00:55:00     67.125        0.0  65.444444   0.003472   
 3        3 2012-03-01 00:55:00     61.500        0.0  62.444444   0.003472   
 4        4 2012-03-01 00:55:00     66.875        0.0  64.444444   0.003472   
 
    x_t-9_d0  x_t-9_d1  x_t-8_d0  x_t-8_d1  ...  y_t+8_d0  y_t+8_d1  y_t+9_d0  \
 0     64.00  0.006944       0.0  0.010417  ...    63.125  0.065972    62.125   
 1     63.75  0.006944       0.0  0.010417  ...    63.875  0.065972    61.250   
 2     60.00  0.006944       0.0  0.010417  ...    65.375  0.065972    60.750   
 3     59.00  0.006944       0.0  0.010417  ...    60.250  0.065972    61.375   
 4     66.50  0.006944       0.0  0.010417  ...    62.500  0.065972    59.125   
 
    y_t+9_d1  y_t+10_d0  y_t+10_d1  

In [ ]:
df["t0_timestamp"].isna().mean(), df["node_id"].nunique(), df["node_id"].min(), df["node_id"].max()
df["t0_timestamp"].min(), df["t0_timestamp"].max()


(Timestamp('2012-03-01 00:55:00'), Timestamp('2012-03-04 09:25:00'))

# Check for Duplicates

In [ ]:
dupe_rate = df.duplicated(["node_id", "t0_timestamp"]).mean()
dupe_rate

np.float64(0.0)

In [ ]:
na = df.isna().mean().sort_values(ascending=False)
na.head(25)


,0
node_id,0.0
t0_timestamp,0.0
x_t-11_d0,0.0
x_t-11_d1,0.0
x_t-10_d0,0.0
x_t-10_d1,0.0
x_t-9_d0,0.0
x_t-9_d1,0.0
x_t-8_d0,0.0
x_t-8_d1,0.0


In [ ]:
nodes_per_t = df.groupby("t0_timestamp")["node_id"].nunique()
nodes_per_t.describe()


,node_id
count,967.000000
mean,206.825233
std,5.434674
min,38.000000
25%,207.000000
50%,207.000000
75%,207.000000
max,207.000000


# missing nodes
So we see from the previous block that at least one timestep has 38 nodes, whereas the majority of them have 207 nodes. Our next step is to determine which timestamps are missing nodes.

Upon inspection it looks like this may just be a sampling artifact, so i will proceed with checking the entire dataset

In [ ]:
nodes_per_t = df.groupby("t0_timestamp")["node_id"].nunique()

bad = nodes_per_t[nodes_per_t < 207].sort_values()  # ascending: worst first
bad.head(20)   # show the most incomplete timestamps


,node_id
t0_timestamp,
2012-03-04 09:25:00,38


# After an initial evaluation, We will perform a full scan for nan values

In [ ]:
def nan_scan(split="train", cols_keep=None, max_rows=None):
    it = ds[split].to_iterable_dataset()
    cols = cols_keep if cols_keep is not None else ds[split].column_names

    nan_counts = Counter()
    total_rows = 0

    for row in tqdm(it, desc=f"Scanning {split}", total=max_rows):
        total_rows += 1

        for c in cols:
            v = row[c]
            # only floats can be NaN; ints/strings won't be
            if isinstance(v, float) and math.isnan(v):
                nan_counts[c] += 1

        if max_rows is not None and total_rows >= max_rows:
            break

    return total_rows, nan_counts

# common: check only the measurement/value channel
d0_cols = [c for c in ds["train"].column_names if c.endswith("_d0")]

rows_scanned, nans = nan_scan("train", cols_keep=d0_cols)
rows_scanned, nans.most_common(10)


Scanning train: 0it [00:00, ?it/s]

(4962618, [])